# task_01

- https://pages.cs.wisc.edu/~remzi/OSTEP/cpu-api.pdf

In Unix-based operating systems, `fork()` is used to create a new process, which is called the child process. The new process created by `fork()` is a duplicate of the current process, except for the returned value.

Here's how the program will work:

1. Define a variable `x` and set its value to 100 in the main (parent) process.
2. Call `fork()` to create a child process.
3. In the child process, we'll check the value of `x`.
4. Both the child and parent processes will then change the value of `x` and we'll observe what happens.

Here's the Python code:

```python
import os

def main():
    x = 100
    pid = os.fork()

    if pid == 0:  # This is the child process
        print("Child: x = ", x)
        x = 200
        print("Child changed x to ", x)
    else:  # This is the parent process
        print("Parent: x = ", x)
        x = 300
        print("Parent changed x to ", x)

if __name__ == "__main__":
    main()
```

When you run this program on a Unix-based system, you should see the following:

- The parent process will print its version of `x` (100), then change it to 300.
- The child process will print its version of `x` (also 100, as it's a copy of the parent's memory space), then change it to 200.

Each process (parent and child) has its own copy of the variable `x`, so changes in one process do not affect the variable in the other process. This is due to the copy-on-write mechanism used by `fork()` in most Unix-based operating systems.

The initial **parent** process in the context of a program using `os.fork()` is the process that executes the `fork()` system call. In a typical scenario, this is the process that you start when you run your Python script.

To clarify:

1. **Before `os.fork()` is Called**: When you execute a Python script, the operating system creates a process to run this script. This process is what we refer to as the parent process. It's the original process that begins the execution of your code.

2. **At the Moment of `os.fork()` Execution**: The line `pid = os.fork()` in your script is a critical point. When the Python interpreter reaches this line, the `fork()` system call is executed. This call results in the creation of a new process, which is a duplicate of the current process (the parent).

3. **After `os.fork()` is Called**: Once `fork()` has been called, there are now two processes in execution:

   - The **Parent Process**: This is the original process that initiated the `fork()`. It continues executing the subsequent lines of code following the `fork()` call. In your script, this process will receive the PID of the child process as the return value of `fork()`, which is a positive non-zero integer.
   
   - The **Child Process**: This is the new process created by `fork()`. It is an almost exact copy of the parent process at the time of the `fork()` call. This process will receive a return value of `0` from the `fork()` call, signifying that it is the child.

So, in the context of your Python script, the initial parent process is the process running your script from the start until the `fork()` call, after which it becomes one of two processes (the parent and the child) running parallelly.


This distinction is crucial for the processes to know their roles and execute the corresponding code. In a typical `fork()` use case, after the fork, you have two processes running the same code, and they can use the PID value returned by `fork()` to decide which part of the code to execute. If `pid == 0`, the process executes the child-specific code; if `pid > 0`, it executes the parent-specific code.


The use of zero as the return value of `fork()` in the child process is a convention in Unix-like operating systems. This design choice is made for a few reasons:

1. **Uniqueness and Simplicity**: Zero is a unique value that can unambiguously indicate the child process. In the context of process IDs (PIDs), all valid PIDs are positive integers. Therefore, using zero, which is not a valid PID, clearly differentiates the child process inside the `fork()` call. It's a simple and effective way to distinguish the child process from the parent.

2. **Error Handling**: In Unix-like systems, system calls typically return a non-zero value to indicate an error. Since positive integers are used for valid PIDs of the parent process, and negative values could be used to indicate an error, zero becomes a convenient and logical choice for indicating success in the child process. It signifies that `fork()` has successfully created a child process without any error.

3. **Coding Clarity**: Using zero for the child process simplifies the code that follows `fork()`. It allows for clear, readable, and straightforward conditional statements to control the flow of the program. For example, `if (pid == 0)` is immediately recognizable as the code path for the child process.

In summary, using zero as the return value in the child process is a convention rooted in the design principles of Unix-like systems, aiming for simplicity, clear error handling, and ease of understanding in programming.


# task_02

Let's break down what the program will do:

1. **Open a File**: The program first opens a file using the `open()` system call. This call returns a file descriptor, which is an integer that uniquely identifies an open file for the process.

2. **Call `fork()`**: After opening the file, the program calls `fork()` to create a new process.

3. **Access by Child and Parent**: Both the child and parent processes can access the file descriptor returned by `open()`. Since `fork()` duplicates the process, the child process inherits file descriptors from the parent process, including the one associated with the opened file.

4. **Concurrent Writing**: If both processes (parent and child) attempt to write to the file at the same time, their outputs might interleave. This means that the data written to the file by the parent and child processes might be mixed in a non-deterministic manner, leading to a potential data inconsistency or corruption, unless properly managed.

Here's an example Python program that demonstrates this:

```python
import os
import time

def write_to_file(file, text):
    os.write(file, text.encode())
    time.sleep(1)  # Adding a delay to simulate concurrent writing

def main():
    # Open a file
    file_path = "example.txt"
    file = os.open(file_path, os.O_CREAT | os.O_WRONLY)

    # Fork a new process
    pid = os.fork()

    if pid == 0:  # Child process
        write_to_file(file, "Child writing...\n")
    else:  # Parent process
        write_to_file(file, "Parent writing...\n")

    os.close(file)

if __name__ == "__main__":
    main()
```

This script opens a file, then forks a new process. Both the parent and the child processes write to the file. The `write_to_file` function includes a delay to simulate the scenario where both processes attempt to write to the file around the same time.

In practice, when two processes write to the same file simultaneously without any coordination mechanism (like locks), the resulting file content might not be what you expect. To avoid such issues, you typically need to implement some form of synchronization, such as file locks, to ensure that only one process writes to the file at a time.

The code structure executes the writing in a sequential manner in the context of each individual process. However, since fork() creates a separate child process, the parent and child processes run independently and concurrently at the system level. This means that while the code in each process executes sequentially, the parent and child processes themselves are running in parallel.

# task_03

Ensuring that the child process prints "hello" before the parent process prints "goodbye" without using `wait()` in the parent can be challenging. The `wait()` system call is the standard method to ensure that a parent process waits for its child processes to complete.

One approach to achieve this without using `wait()` is by introducing a delay in the parent process. This isn't a reliable method because the timing isn't guaranteed, but it can work in a simple scenario like this one. 

Here's a Python program demonstrating this idea:

```python
import os
import time

pid = os.fork()

if pid == 0:
    # Child process
    print("hello")
else:
    # Parent process
    time.sleep(1)  # Introduce a delay
    print("goodbye")
```

In this script:

- The `fork()` call creates a child process.
- If `pid` is `0`, it means the current process is the child process, which prints "hello".
- In the parent process (where `pid` is non-zero), a `time.sleep(1)` introduces a delay before printing "goodbye".

This delay increases the likelihood that the child process will complete its printing task before the parent process does. However, this method is not foolproof as process scheduling can vary based on several factors like system load and operating system scheduling policies.

For a more robust solution, proper synchronization mechanisms (like pipes, semaphores, or using `wait()`) are recommended. These mechanisms can provide more deterministic control over the order of execution in concurrent processes.

# task_04

Writing a program that calls `fork()` followed by various forms of `exec()` to run `/bin/ls` is an excellent way to understand the different flavors of the `exec()` family of functions in Unix-based systems. Each variant of `exec()` provides a different way to specify the program to be executed and its arguments.

Let's create a Python program that demonstrates each variant:

1. **`execl()`**: Executes a program, specifying the path and arguments as separate arguments.

2. **`execle()`**: Similar to `execl()`, but also allows specifying the environment for the new program.

3. **`execlp()`**: Searches for the program in the system's PATH and takes the program name and arguments as separate arguments.

4. **`execv()`**: Executes a program, specifying the path and arguments as a list.

5. **`execvp()`**: Searches for the program in the system's PATH and takes the program name and arguments as a list.

6. **`execvpe()`**: Similar to `execvp()`, but also allows specifying the environment for the new program.

Here's the Python code:

```python
import os

def exec_example(command, variant, env=None):
    pid = os.fork()
    if pid == 0:  # Child process
        if variant == 'execl':
            os.execl('/bin/ls', 'ls', '-l')
        elif variant == 'execle':
            os.execle('/bin/ls', 'ls', '-l', env)
        elif variant == 'execlp':
            os.execlp('ls', 'ls', '-l')
        elif variant == 'execv':
            os.execv('/bin/ls', ['ls', '-l'])
        elif variant == 'execvp':
            os.execvp('ls', ['ls', '-l'])
        elif variant == 'execvpe':
            os.execvpe('ls', ['ls', '-l'], env)
        os._exit(0)  # Exit child process after exec call

# Specify environment for execle and execvpe
env = os.environ.copy()

# Test each variant
for variant in ['execl', 'execle', 'execlp', 'execv', 'execvp', 'execvpe']:
    exec_example('/bin/ls', variant, env)
```

In this script, the function `exec_example` is used to demonstrate each variant. The `fork()` system call creates a new child process, and depending on the variant, one of the `exec()` family functions is called to execute `/bin/ls`.

### Why So Many Variants?

The reason for the existence of so many variants of `exec()` is to provide flexibility in how programs are executed:

- **Different Argument Formats**: Some variants allow specifying arguments as a list (e.g., `execv`), while others take arguments directly (e.g., `execl`).
- **Environment Control**: Some variants allow specifying a new environment for the executed program (e.g., `execle`, `execvpe`).
- **Path Resolution**: The `p` variants (`execlp`, `execvp`, `execvpe`) automatically search the system's PATH environment variable to find the executable, which is convenient when the path of the executable isn't known in advance.

This flexibility allows programmers to choose the variant that best fits their specific requirements and scenarios.

# task_05

To write a program where the parent process uses `wait()` to wait for the child process to finish, we can create a simple Python script using the `os.fork()` and `os.wait()` functions. The `wait()` function in Unix-based systems is used by a parent process to wait for the state change in a child process - typically, this means waiting for the child process to terminate.

Here's what the program will do:

1. The parent process will call `fork()` to create a child process.
2. The child process will perform some task (e.g., print a message).
3. The parent process will call `wait()`, which will block until the child process completes.
4. `wait()` returns a tuple containing the PID of the terminated child process and an integer indicating the child's exit status.

Let's see this in code:

```python
import os

pid = os.fork()

if pid == 0:
    # Child process
    print("Hello from the child process!")
    os._exit(0)  # Terminate child process
else:
    # Parent process
    print("Parent process is waiting for the child to finish.")
    finished_pid, status = os.wait()
    print(f"Child process with PID {finished_pid} finished with status {status}")

    # Optional: Decoding the exit status
    exit_status = os.WEXITSTATUS(status)
    print(f"Decoded exit status: {exit_status}")
```

In this script:

- The child process prints a message and then exits.
- The parent process waits for the child to finish using `os.wait()`.
- `os.wait()` returns a tuple containing the PID of the finished child and its exit status. The exit status is a 16-bit number whose lower byte indicates the signal number that killed the process (if non-zero), and whose upper byte indicates the exit status (if the signal number is zero).

If `wait()` is used in the child process, it will typically return an error, since there are no child processes to wait for (assuming the child hasn't created any child processes of its own). In Python, it will raise an `OSError`. This is because `wait()` is meant to be used by parent processes to wait for their child processes.

# task_06

To modify the previous program to use `waitpid()` instead of `wait()`, we'll make a small change where the parent process explicitly waits for the termination of the specific child process identified by its PID. The `waitpid()` function is more versatile than `wait()`, as it allows the parent process to wait for a specific child process to change state, rather than any child process.

Here's the modified Python program using `waitpid()`:

```python
import os

pid = os.fork()

if pid == 0:
    # Child process
    print("Hello from the child process!")
    os._exit(0)  # Terminate child process
else:
    # Parent process
    print("Parent process is waiting for the specific child to finish.")
    finished_pid, status = os.waitpid(pid, 0)  # Wait for the specific child
    print(f"Child process with PID {finished_pid} finished with status {status}")

    # Optional: Decoding the exit status
    exit_status = os.WEXITSTATUS(status)
    print(f"Decoded exit status: {exit_status}")
```

In this script, `os.waitpid(pid, 0)` is used in the parent process to wait for the child process with PID `pid` to change state (in this case, to terminate).

### When is `waitpid()` Useful?

1. **Waiting for a Specific Child**: Unlike `wait()`, which waits for any child process to finish, `waitpid()` can be used to wait for a specific child process. This is useful in situations where a parent process creates multiple child processes and needs to wait for a particular one.

2. **Non-blocking Wait**: With certain options, `waitpid()` can be used in a non-blocking manner. This means the parent process can continue executing while periodically checking the status of a child process. This is done by using the `WNOHANG` option as the second argument.

3. **Additional Control**: `waitpid()` offers more control over different child states beyond just termination. For example, it can detect if a child process has stopped executing (but not terminated), which can be useful for implementing certain types of process control in more complex applications.

4. **Error Handling**: `waitpid()` allows for more detailed error handling since you can check the status of a specific child process and respond accordingly.

In summary, `waitpid()` is a more flexible and powerful tool compared to `wait()`, particularly in scenarios involving multiple child processes or when specific child process monitoring and control are required.

# task_07

In C, `printf()` function is used for printing to stdout, but in Python, we typically use `print()`. For this demonstration, I'll use Python's `sys.stdout` and `os` modules to emulate this behavior.

Here's what the program will do:

1. Create a child process using `fork()`.
2. In the child process, close the standard output file descriptor.
3. Attempt to print something from the child process.

Let's see the Python code for this:

```python
import os
import sys

pid = os.fork()

if pid == 0:
    # Child process
    os.close(sys.stdout.fileno())  # Close standard output
    print("This print statement is after closing stdout.")
    os._exit(0)  # Exit child process
else:
    # Parent process
    os.wait()  # Wait for the child process to finish
    print("Child process has finished.")
```

In this script:

- The child process closes its standard output using `os.close(sys.stdout.fileno())`. The `fileno()` method returns the file descriptor associated with `sys.stdout` (which is typically 1 for standard output).
- After closing stdout, the child process attempts to print a message. However, since its standard output is closed, this message will not appear on the console. In a C program, if `printf()` is called after closing stdout, it will fail to print to the terminal for the same reason.

Closing a file descriptor like standard output means that the process no longer has a valid way to write to the standard output stream. Any attempts to write to it will not have the usual effect (i.e., displaying text on the terminal). Depending on the programming language and environment, this might result in an error or simply no visible output. In Python, you may also see an exception depending on how the environment handles writes to a closed file descriptor.

# task_08

Creating a program that creates two child processes and connects the standard output of one to the standard input of the other using a pipe is a classic example of inter-process communication in Unix-like systems. This is achieved using the `pipe()` system call to create a pipe and then forking two child processes. One child process writes to the pipe, and the other reads from it.

Here's how you can implement this in Python:

1. Create a pipe using `os.pipe()`.
2. Fork the first child process to write to the pipe.
3. Fork the second child process to read from the pipe.
4. In the first child, redirect standard output to the write end of the pipe.
5. In the second child, redirect standard input to the read end of the pipe.

Here's the Python code to illustrate this:

```python
import os

def child1(pipeout):
    os.close(pipeout)  # Close unused read end
    os.dup2(pipeout, 1)  # Duplicate write end to stdout
    os.execlp("echo", "echo", "Hello from child 1")

def child2(pipein):
    os.close(pipein)  # Close unused write end
    os.dup2(pipein, 0)  # Duplicate read end to stdin
    os.execlp("cat", "cat")

def main():
    pipein, pipeout = os.pipe()  # Create a pipe

    if os.fork() == 0:
        child1(pipeout)  # First child process

    if os.fork() == 0:
        child2(pipein)  # Second child process

    # In parent process
    os.close(pipein)   # Close the read end
    os.close(pipeout)  # Close the write end
    os.wait()
    os.wait()

if __name__ == "__main__":
    main()
```

In this script:

- We create a pipe with `os.pipe()`, which returns two file descriptors: `pipein` (for reading) and `pipeout` (for writing).
- We then create two child processes using `os.fork()`.
- In the first child (`child1`), we close the unused read end of the pipe and use `os.dup2()` to duplicate the pipe's write end to standard output (`stdout`). Then we execute a command (`echo`) to send data into the pipe.
- In the second child (`child2`), we close the unused write end of the pipe and duplicate the pipe's read end to standard input (`stdin`). Then we execute a command (`cat`) to read data from the pipe.
- In the parent process, we close both ends of the pipe and wait for both child processes to complete.

This program demonstrates a simple but effective form of inter-process communication using pipes, where output from one process is directly fed as input to another.